# Import

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
#import seaborn as sns
import seaborn as sns
sns.set_theme(style="dark")
import matplotlib.pyplot as plt
import os
from shutil import copyfile
#import splitfolders
from sklearn.model_selection import train_test_split

In [3]:
from neuralart.data import *

# Download Chan data

# Get Chan data

In [4]:
chan_csv_folder_path = "../raw_data/wikiart/csv_chan"
chan_image_folder_path= "../raw_data/wikiart/dataset_chan"
csv_output_path= "../raw_data/wikiart/test"

In [5]:
chan_data = get_chan_data(chan_csv_folder_path, chan_image_folder_path, csv_output_path=csv_output_path)
chan_data.shape

(78748, 10)

In [6]:
chan_data

,image_name,style,genre,artist,title,chan_image_path,chan_image_name,chan_split_style,chan_split_genre,chan_split_artist
0,pointillism_henri-edmond-cross_chaine-des-maur...,pointillism,landscape,henri-edmond-cross,chaine-des-maures-1907.jpg,Pointillism/henri-edmond-cross_chaine-des-maur...,henri-edmond-cross_chaine-des-maures-1907.jpg,train,train,NaN
1,pointillism_theo-van-rysselberghe_fountain-at-...,pointillism,cityscape,theo-van-rysselberghe,fountain-at-san-souci-1903.jpg,Pointillism/theo-van-rysselberghe_fountain-at-...,theo-van-rysselberghe_fountain-at-san-souci-19...,train,train,NaN
2,pointillism_georges-seurat_the-away-samson-in-...,pointillism,landscape,georges-seurat,the-away-samson-in-grandcamp-1885.jpg,Pointillism/georges-seurat_the-away-samson-in-...,georges-seurat_the-away-samson-in-grandcamp-18...,train,train,NaN
3,pointillism_theo-van-rysselberghe_portrait-of-...,pointillism,portrait,theo-van-rysselberghe,portrait-of-madame-charles-maus-1890.jpg,Pointillism/theo-van-rysselberghe_portrait-of-...,theo-van-rysselberghe_portrait-of-madame-charl...,train,train,NaN
4,pointillism_jan-toorop_broek-in-waterland-1889...,pointillism,landscape,jan-toorop,broek-in-waterland-1889.jpg,Pointillism/jan-toorop_broek-in-waterland-1889...,jan-toorop_broek-in-waterland-1889.jpg,train,train,NaN
...,...,...,...,...,...,...,...,...,...,...
81441,art-nouveau-modern_boris-kustodiev_mercahnt-s-...,art_nouveau_modern,genre_painting,boris-kustodiev,mercahnt-s-wife-with-purchases-1920.jpg,Art_Nouveau_Modern/boris-kustodiev_mercahnt-s-...,boris-kustodiev_mercahnt-s-wife-with-purchases...,val,val,val
81442,art-nouveau-modern_zinaida-serebriakova_self-p...,art_nouveau_modern,NaN,zinaida-serebriakova,self-portrait-1907.jpg,Art_Nouveau_Modern/zinaida-serebriakova_self-p...,zinaida-serebriakova_self-portrait-1907.jpg,val,NaN,NaN
81443,art-nouveau-modern_nicholas-roerich_pskov-chur...,art_nouveau_modern,cityscape,nicholas-roerich,pskov-churchyard-1904-1.jpg,Art_Nouveau_Modern/nicholas-roerich_pskov-chur...,nicholas-roerich_pskov-churchyard-1904-1.jpg,train,train,train
81444,art-nouveau-modern_koloman-moser_poster-for-fe...,art_nouveau_modern,NaN,koloman-moser,poster-for-felix-salzens-jung-wiener-theatre-w...,Art_Nouveau_Modern/koloman-moser_poster-for-fe...,koloman-moser_poster-for-felix-salzens-jung-wi...,train,NaN,NaN


In [7]:
merge_mov_1={'name': 'style_m1',
            'merging':{'abstract_expressionism': 'abstract', 'action_painting': 'abstract', 'analytical_cubism': 'cubism', 
                       'art_nouveau_modern': None, 'baroque': None, 'color_field_painting': 'color_field_painting', 
                       'contemporary_realism': None, 'cubism': 'cubism', 'early_renaissance': 'renaissance', 'expressionism': 'expressionism',
                       'fauvism': None, 'high_renaissance': 'renaissance', 'impressionism': 'impressionism',
                       'mannerism_late_renaissance': None, 'minimalism': None, 'naive_art_primitivism': None,
                       'new_realism': None, 'northern_renaissance': 'renaissance', 'pointillism': None, 'pop_art': None,
                       'post_impressionism': None, 'realism': 'realism', 'rococo': None, 'romanticism': 'romanticism',
                       'symbolism': None, 'synthetic_cubism': 'cubism', 'ukiyo_e': None}}

csv_output_path =  "../raw_data/wikiart/test"
image_folder_output_path = "../raw_data/wikiart/test"

train_ratio=0.8
val_ratio=0.2
test_ratio=0.1

In [8]:
data=create_dataset(chan_data, target="style", merge=merge_mov_1, n=100, strategy='drop', keep_genre=False, random_state=123, flat=False, 
                      chan_image_folder_path=chan_image_folder_path, csv_output_path=csv_output_path, image_folder_output_path=image_folder_output_path,
                      val_ratio=val_ratio,test_ratio=test_ratio)

/home/gregoirelafay/.pyenv/versions/3.8.6/envs/neuralart/lib/python3.8/site-packages/pandas/core/frame.py:3640: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = value


Done: 800 image(s) copied, 3 image(s) in the folder


In [ ]:
path = '/home/User/Documents/file.txt'
os.path.dirname(path)

In [ ]:
data_sample = get_dataset(data, target="movement", class_=merge_mov_1,random_state=123, keep_genre=True, output_path=output_path)


# Create data directory

In [ ]:
create_dataset_directory(data_sample, chan_image_path, output_path, 'wikiart-movement-genre_True-class_8-merge_mov-1', flat=False)


In [ ]:
input_dir = os.path.join(output_path,"wikiart-movement-genre_True-class_8-merge_mov-1")
output_dir = os.path.join(output_path,"train_val_test_True_all")
csv_path = os.path.join(output_path,"wikiart-movement-genre_True-class_8-merge_mov-1.csv")

In [ ]:
get_train_val_test_directory(output_dir, input_dir=input_dir,
                                 csv_path=csv_path,
                                 save_csv=True,
                                 train_ratio=0.8,
                                 val_ratio=0.1,
                                 test_ratio=0.1,
                                 seed=1337)

# Data Visualization

In [ ]:
data = pd.read_csv("wikiart-movement-genre_True-class_8-merge_mov-1_split.csv")

In [ ]:
col = ['artist','genre','movement']

fig, ax = plt.subplots(1,1,figsize=(15,10))
sns.barplot(y=data[col].nunique().index, 
            x=data[col].nunique().values,
            order=data[col].nunique().sort_values(ascending=False).index,
            ax=ax);

for i, v in enumerate(data[col].nunique().sort_values(ascending=False).values):
    ax.text(v, i , str(v), color='blue', fontweight='bold')
    
ax.set_title("Number of classes per target");
ax.set_xlabel("Count");
ax.set_ylabel("Target");

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(1,1,figsize=(15,15));
sns.countplot(data=data, y="movement", 
              order = data['movement'].value_counts().index,
              ax=ax);
sns.despine()

for i, v in enumerate(data["movement"].value_counts().values):
    ax.text(v, i , str(v), color='blue', fontweight='bold',fontsize='25',horizontalalignment='right')
    
ax.set_title("Number of images per movement",color="#FF4500",size=30);
ax.set_ylabel("Movement (Art Style)");
ax.spines["bottom"].set_color("#FF4500")
ax.spines["left"].set_color("#FF4500")
ax.tick_params(axis='y', colors='#FF4500',labelsize=26)
ax.set_ylabel("Art Style",
                  loc='center',
                  rotation='vertical',
                  color="#FF4500",size=20)
fig.patch.set_alpha(0)
ax.patch.set_alpha(0)

fig.savefig('class_split_2.png',bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10));
sns.countplot(data=data, y="genre", 
              order = data['genre'].value_counts().index,
              ax=ax);

for i, v in enumerate(data["genre"].value_counts().values):
    ax.text(v, i , str(v), color='blue', fontweight='bold')
    
ax.set_title("Number of images per genre");

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10));
sns.barplot(y=data.groupby("movement").artist.nunique().index, 
            x=data.groupby("movement").artist.nunique().values,
            order=data.groupby("movement").artist.nunique().sort_values(ascending=False).index,
            ax=ax);

for i, v in enumerate(data.groupby("movement").artist.nunique().sort_values(ascending=False).values):
    ax.text(v, i , str(v), color='blue', fontweight='bold')
    
ax.set_title("Number of artists per movement");
ax.set_xlabel("Count");